In [2]:
import numpy as np
import pandas as pd

In [33]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [4]:
Corpus = pd.read_csv("train.csv")

In [5]:
Corpus.shape
Corpus.info()
Corpus.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
Corpus_WO_Label = Corpus.drop('label', axis=1)
Corpus_Label = Corpus['label']

In [7]:
Corpus_Label.value_counts()

1    10413
0    10387
Name: label, dtype: int64

In [8]:
print(Corpus['text'])

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        Ever get the feeling your life circles the rou...
2        Why the Truth Might Get You Fired October 29, ...
3        Videos 15 Civilians Killed In Single US Airstr...
4        Print \nAn Iranian woman has been sentenced to...
                               ...                        
20795    Rapper T. I. unloaded on black celebrities who...
20796    When the Green Bay Packers lost to the Washing...
20797    The Macy’s of today grew from the union of sev...
20798    NATO, Russia To Hold Parallel Exercises In Bal...
20799      David Swanson is an author, activist, journa...
Name: text, Length: 20800, dtype: object


In [9]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
# Pandas function to remove blank rows , if inplace is set to true, dataframe is updated without returning any value
Corpus['text'].dropna(inplace=True)
# Convert text to lowercase since python is case senstive
Corpus['text'] = [str(text).lower() for text in Corpus['text']]
# In this each text in Corpus will be broken into set of words
Corpus['text']= [word_tokenize(str(text)) for text in Corpus['text']]

[nltk_data] Downloading package punkt to /Users/humaid/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/humaid/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/humaid/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/humaid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# Creating a dictionary of pos_tags to understand the type of word whether it is noun / adjective / adverb / verb. 
# Default value of pos_tags is noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['R'] = wn.ADV
tag_map['V'] = wn.VERB
tag_map['J'] = wn.ADJ

In [11]:
for index,text in enumerate(Corpus['text']):
    final_words = []
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function provides whether the word is Noun/Verb/Adverb/Adjective or something else.
    for word, tag in pos_tag(text):
        # To remove stop words and consider only alphabetical words
        if word not in stopwords.words('english') and word.isalpha():
            final_word = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            final_words.append(final_word)
    # After each iteration, final_words will be stored in corpus
    Corpus.loc[index,'text_final'] = str(final_words)

In [12]:
Train_Corpus_WO_Label, Val_Test_Corpus_WO_Label, Train_Label, Val_Test_Label = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.4)
Validation_Corpus_WO_Label, Test_Corpus_WO_Label, Validation_Label, Test_Label = model_selection.train_test_split(Val_Test_Corpus_WO_Label,Val_Test_Label,test_size=0.5)

In [13]:
Tfidf_vector = TfidfVectorizer(max_features=6000)
Tfidf_vector.fit(Corpus['text_final'])
Train_Corpus_WO_Label_Tfidf = Tfidf_vector.transform(Train_Corpus_WO_Label)
Validation_Corpus_WO_Label_Tfidf = Tfidf_vector.transform(Validation_Corpus_WO_Label)

In [22]:
print(Tfidf_vector.vocabulary_)

{'house': 2560, 'dem': 1408, 'aide': 142, 'even': 1885, 'see': 4735, 'comey': 1016, 'letter': 3090, 'jason': 2873, 'chaffetz': 835, 'tweet': 5558, 'october': 3673, 'subscribe': 5152, 'american': 202, 'utah': 5676, 'image': 2626, 'courtesy': 1232, 'michael': 3363, 'available': 404, 'creative': 1251, 'license': 3105, 'apology': 259, 'keith': 2949, 'doubt': 1627, 'bad': 430, 'person': 3928, 'world': 5902, 'director': 1527, 'james': 2864, 'accord': 36, 'democratic': 1413, 'look': 3164, 'like': 3114, 'also': 187, 'know': 2986, 'well': 5834, 'turn': 5554, 'send': 4751, 'announce': 233, 'fbi': 2038, 'email': 1749, 'may': 3295, 'relate': 4426, 'hillary': 2501, 'clinton': 948, 'server': 4772, 'ranking': 4323, 'democrat': 1412, 'relevant': 4435, 'committee': 1032, 'hear': 2459, 'find': 2090, 'via': 5719, 'one': 3704, 'republican': 4479, 'chairman': 838, 'notify': 3625, 'rank': 4322, 'member': 3328, 'intelligence': 2772, 'judiciary': 2922, 'oversight': 3786, 'agency': 127, 'review': 4541, 'recent

In [15]:
nb = naive_bayes.MultinomialNB()
nb.fit(Train_Corpus_WO_Label_Tfidf,Train_Label)
# Validation dataset used for predicting labels
predictions_NB = nb.predict(Validation_Corpus_WO_Label_Tfidf)
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Validation_Label)*100)

Naive Bayes Accuracy Score ->  87.11538461538461


In [47]:
SVM_Kernels = ['linear', 'poly', 'rbf', 'sigmoid']
SVM_Gammas = ['scale','auto']
SVM_C = [1.0,2.0,3.0,4.0,5.0]
SVM_Degrees = [2,3,4,5]
SVM_Prediction = np.empty((0,8))
for svm_kernel in SVM_Kernels:
    for svm_gamma in SVM_Gammas:
        for svm_degree in SVM_Degrees:
            for svm_c in SVM_C:
                if svm_kernel != 'poly' and svm_degree != 2:
                    continue
                SVM = svm.SVC(C=svm_c, kernel=svm_kernel, gamma=svm_gamma,degree=svm_degree)
                SVM.fit(Train_Corpus_WO_Label_Tfidf,Train_Label)
                predictions_SVM = SVM.predict(Validation_Corpus_WO_Label_Tfidf)
                print(svm_kernel , svm_gamma , svm_c, svm_degree,"SVM F1 Score -> ",f1_score(Validation_Label,predictions_SVM)*100)
                SVM_Prediction = np.append(SVM_Prediction, np.array([[svm_kernel, svm_gamma, svm_c, svm_degree,accuracy_score(Validation_Label,predictions_SVM)*100,precision_score(Validation_Label, predictions_SVM)*100,recall_score(Validation_Label, predictions_SVM)*100,f1_score(Validation_Label,predictions_SVM)*100]]), axis=0)



linear scale 1.0 2 SVM F1 Score ->  93.30432680686489
linear scale 2.0 2 SVM F1 Score ->  93.08510638297872
linear scale 3.0 2 SVM F1 Score ->  92.95910960561336
linear scale 4.0 2 SVM F1 Score ->  92.51569290197973
linear scale 5.0 2 SVM F1 Score ->  92.37063778580024
linear auto 1.0 2 SVM F1 Score ->  93.30432680686489
linear auto 2.0 2 SVM F1 Score ->  93.08510638297872
linear auto 3.0 2 SVM F1 Score ->  92.95910960561336
linear auto 4.0 2 SVM F1 Score ->  92.51569290197973
linear auto 5.0 2 SVM F1 Score ->  92.37063778580024
poly scale 1.0 2 SVM F1 Score ->  92.98329355608591
poly scale 2.0 2 SVM F1 Score ->  93.13959028108623
poly scale 3.0 2 SVM F1 Score ->  93.04533586517921
poly scale 4.0 2 SVM F1 Score ->  93.01775147928993
poly scale 5.0 2 SVM F1 Score ->  92.89256198347108
poly scale 1.0 3 SVM F1 Score ->  90.87885985748217
poly scale 2.0 3 SVM F1 Score ->  91.11111111111111
poly scale 3.0 3 SVM F1 Score ->  90.97320169252468
poly scale 4.0 3 SVM F1 Score ->  91.007025761124

In [44]:
from sklearn.linear_model import LogisticRegression
 
Switch_LR_penalty ={
'newton-cg' : ['l2', 'none'],
'lbfgs' : ['l2', 'none'],
'liblinear' : ['l1', 'l2'],
'sag' : ['l2', 'none'],
'saga'     : ['l1', 'l2', 'none']
}
LR_C_list = [1.0,2.0,3.0,4.0,5.0]
LR_MAX_ITER_list = [250,500,1000,1500,2500]
LR_Solver_list = ['liblinear', 'saga','sag','newton-cg', 'lbfgs']
LR_Prediction = np.empty((0,8))
 
for lr_solver in LR_Solver_list:
    LR_Penalty_list = Switch_LR_penalty.get(lr_solver, -1)
    for lr_penalty in LR_Penalty_list:
        for lr_c in LR_C_list:
            for lr_max_iter in LR_MAX_ITER_list:
                Logistic_Reg = LogisticRegression(verbose=1, solver=lr_solver,random_state=0, C=lr_c, penalty=lr_penalty,max_iter=lr_max_iter)
                model=Logistic_Reg.fit(Train_Corpus_WO_Label_Tfidf,Train_Label)
                predictions_Logreg = Logistic_Reg.predict(Validation_Corpus_WO_Label_Tfidf)
                print(lr_solver , lr_penalty , lr_c, lr_max_iter,"Logistic Regression F1 Score -> ",f1_score(Validation_Label,predictions_Logreg)*100)
                LR_Prediction = np.append(LR_Prediction, np.array([[lr_solver,lr_penalty, lr_c, lr_max_iter,accuracy_score(Validation_Label,predictions_Logreg)*100,precision_score(Validation_Label, predictions_Logreg)*100,recall_score(Validation_Label, predictions_Logreg)*100,f1_score(Validation_Label,predictions_Logreg)*100]]), axis=0)


[LibLinear]liblinear l1 1.0 250 Logistic Regression F1 Score ->  92.48916706788638
[LibLinear]liblinear l1 1.0 500 Logistic Regression F1 Score ->  92.48916706788638
[LibLinear]liblinear l1 1.0 1000 Logistic Regression F1 Score ->  92.48916706788638
[LibLinear]liblinear l1 1.0 1500 Logistic Regression F1 Score ->  92.48916706788638
[LibLinear]liblinear l1 1.0 2500 Logistic Regression F1 Score ->  92.48916706788638
[LibLinear]liblinear l1 2.0 250 Logistic Regression F1 Score ->  93.56078808265256
[LibLinear]liblinear l1 2.0 500 Logistic Regression F1 Score ->  93.56078808265256
[LibLinear]liblinear l1 2.0 1000 Logistic Regression F1 Score ->  93.56078808265256
[LibLinear]liblinear l1 2.0 1500 Logistic Regression F1 Score ->  93.56078808265256
[LibLinear]liblinear l1 2.0 2500 Logistic Regression F1 Score ->  93.56078808265256
[LibLinear]liblinear l1 3.0 250 Logistic Regression F1 Score ->  93.46781940441883
[LibLinear]liblinear l1 3.0 500 Logistic Regression F1 Score ->  93.4678194044188

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 98 epochs took 4 seconds
saga l1 1.0 250 Logistic Regression F1 Score ->  92.48916706788638


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


convergence after 98 epochs took 4 seconds
saga l1 1.0 500 Logistic Regression F1 Score ->  92.48916706788638


...

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished
/Users/humaid/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


lbfgs none 5.0 1000 Logistic Regression F1 Score ->  92.45147375988498


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished
/Users/humaid/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


lbfgs none 5.0 1500 Logistic Regression F1 Score ->  92.45147375988498
lbfgs none 5.0 2500 Logistic Regression F1 Score ->  92.45147375988498


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s finished


In [46]:
LR_Prediction[np.argmax(LR_Prediction[:,-1])]

array(['liblinear', 'l1', '2.0', '250', '93.5576923076923',
       '93.51585014409221', '93.60576923076923', '93.56078808265256'],
      dtype='<U32')

In [50]:
LR_Prediction[np.argmin(LR_Prediction[:,-1])]

array(['newton-cg', 'none', '1.0', '250', '91.7548076923077',
       '91.7347429120615', '91.77884615384615', '91.75678923335737'],
      dtype='<U32')

In [48]:
SVM_Prediction[np.argmax(SVM_Prediction[:,-1])]

array(['rbf', 'scale', '3.0', '2', '94.13461538461539',
       '94.34782608695652', '93.89423076923077', '94.12048192771084'],
      dtype='<U32')

In [49]:
SVM_Prediction[np.argmin(SVM_Prediction[:,-1])]

array(['poly', 'auto', '1.0', '2', '50.0', '50.0', '100.0',
       '66.66666666666666'], dtype='<U32')

In [18]:
Test_Corpus_WO_Label_Tfidf = Tfidf_vector.transform(Test_Corpus_WO_Label)

In [52]:
# Test dataset used for predicting labels
test_predictions_NB = nb.predict(Test_Corpus_WO_Label_Tfidf)
print("Naive Bayes Accuracy Score -> ",accuracy_score(test_predictions_NB, Test_Label)*100)
print("Naive Bayes Recall Score -> ",recall_score(test_predictions_NB, Test_Label)*100)
print("Naive Bayes Precision Score -> ",precision_score(test_predictions_NB, Test_Label)*100)
print("Naive Bayes F1 Score -> ",f1_score(test_predictions_NB, Test_Label)*100)

Naive Bayes Accuracy Score ->  87.66826923076924
Naive Bayes Recall Score ->  90.99427381572099
Naive Bayes Precision Score ->  83.71647509578544
Naive Bayes F1 Score ->  87.2037914691943


In [57]:
# SVM Classifier
SVM = svm.SVC(C=float(SVM_Prediction[np.argmax(SVM_Prediction[:,-1])][2]), kernel=SVM_Prediction[np.argmax(SVM_Prediction[:,-1])][0], gamma=SVM_Prediction[np.argmax(SVM_Prediction[:,-1])][1])
SVM.fit(Train_Corpus_WO_Label_Tfidf,Train_Label)
test_predictions_SVM = SVM.predict(Test_Corpus_WO_Label_Tfidf)
print("SVM Accuracy Score -> ",accuracy_score(test_predictions_SVM, Test_Label)*100)
print("SVM Recall Score -> ",recall_score(test_predictions_SVM, Test_Label)*100)
print("SVM Precision Score -> ",precision_score(test_predictions_SVM, Test_Label)*100)
print("SVM F1 Score -> ",f1_score(test_predictions_SVM, Test_Label)*100)

SVM Accuracy Score ->  94.32692307692308
SVM Recall Score ->  94.56207892204043
SVM Precision Score ->  94.10919540229885
SVM F1 Score ->  94.33509361497839


In [51]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
 
Logistic_Reg = LogisticRegression(verbose=1, solver=LR_Prediction[np.argmax(LR_Prediction[:,-1])][0],random_state=0, C=float(LR_Prediction[np.argmax(LR_Prediction[:,-1])][2]), penalty=LR_Prediction[np.argmax(LR_Prediction[:,-1])][1],max_iter=int(LR_Prediction[np.argmax(LR_Prediction[:,-1])][3]))
model=Logistic_Reg.fit(Train_Corpus_WO_Label_Tfidf,Train_Label)
test_predictions_Logreg = Logistic_Reg.predict(Test_Corpus_WO_Label_Tfidf)
print("Logistic Regression Accuracy Score -> ",accuracy_score(test_predictions_Logreg, Test_Label)*100)
print("Logistic Regression Recall Score -> ",recall_score(test_predictions_Logreg, Test_Label)*100)
print("Logistic Regression Precision Score -> ",precision_score(test_predictions_Logreg, Test_Label)*100)
print("Logistic Regression F1 Score -> ",f1_score(test_predictions_Logreg, Test_Label)*100)


[LibLinear]Logistic Regression Accuracy Score ->  93.19711538461539
Logistic Regression Recall Score ->  93.1611669057867
Logistic Regression Precision Score ->  93.29501915708812
Logistic Regression F1 Score ->  93.22804498683895
